In [1]:
from jduck.utils.logging_config import logger as logging

In [2]:
logging.info('Hello')

  210308-12:59|<ipython-input-2-1e24994359dc>:1|INFO|  Hello


In [4]:
from jduck.robot import JDuck

ModuleNotFoundError: No module named 'DCMotor'

In [1]:
from jduck import Camera

ImportError: cannot import name 'Camera'